In [1]:
import os
import time
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
from lib.metrics import create_dir
from lib.load_data import get_data

import cv2
from skimage import io
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
#from tqdm.notebook import tqdm

import matplotlib.font_manager
import matplotlib.pyplot as plt
#plt.rcParams["font.family"] = "Times New Roman"

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, model_from_json

print('TensorFlow version: {version}'.format(version=tf.__version__))
print('Keras version: {version}'.format(version=tf.keras.__version__))
print('Eager mode enabled: {mode}'.format(mode=tf.executing_eagerly()))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

2023-06-18 18:59:37.014138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 18:59:37.639498: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mpanta1/.conda/envs/tf_gpu/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


TensorFlow version: 2.12.0
Keras version: 2.12.0
Eager mode enabled: True
Num GPUs Available:  1
Num CPUs Available:  1


In [2]:
class PCALayer(tf.keras.layers.Layer):
    def __init__(self, n_components, **kwargs):
        super(PCALayer, self).__init__(**kwargs)
        self.n_components = n_components

    def build(self, input_shape):
        self.shape = input_shape
        self.input_dim = int(input_shape[-1])
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.input_dim, self.n_components), dtype="float32",
                                      initializer='glorot_uniform',
                                      trainable=False)

    def call(self, x):
        # Flatten the input tensor
        #x = tf.linalg.normalize(x,axis=-1)
        #print(x.shape)
        # assumption is that the feature vector is normalized
        #x = tf.math.l2_normalize(x, axis=-1)
        batch_size = tf.shape(x)[0]
        flattened = tf.reshape(x, [batch_size, -1, self.input_dim])
        
        # Compute the mean and subtract it from the input tensor
        mean = tf.reduce_mean(flattened, axis=1, keepdims=True)
        centered = flattened - mean
        

        # Compute the covariance matrix
        cov = tf.matmul(centered, centered, transpose_a=True) / tf.cast(tf.shape(flattened)[1] - 1, tf.float32)

        # Compute the eigenvalues and eigenvectors of the covariance matrix
        eigenvalues, eigenvectors = tf.linalg.eigh(cov)

        # Sort the eigenvectors based on the eigenvalues
        idx = tf.argsort(eigenvalues, axis=-1, direction='DESCENDING')
        top_eigenvectors = tf.gather(eigenvectors, idx, batch_dims=1, axis=-1)
        top_eigenvectors = top_eigenvectors[:, :, :self.n_components]

        # Transpose the eigenvectors to match the input shape
        top_eigenvectors = tf.transpose(top_eigenvectors, perm=[0, 1, 2])
        
        # Project centered data onto top principal components
        projected = tf.matmul(centered, top_eigenvectors)

        # Reshape projected data and return as output
        output_shape = tf.concat([tf.shape(x)[:-1], [self.n_components]], axis=0)
        #output = tf.reshape(projected, shape=(-1, *self.output_shape))
        output = tf.reshape(projected, output_shape)
        return output



    def compute_output_shape(self, input_shape):
        return tuple(input_shape[:-1]) + (self.n_components,)

    def get_config(self):
        config = super(PCALayer, self).get_config()
        config.update({'n_components': self.n_components})
        return config



In [3]:

def prediction(test_ids, image_path, mask_path, model_dict):
    # empty list to store results
    image_names, image_id, mask_id, has_mask = [], [], [], []

    results_dict = {name: [] for name in model_dict.keys()}
    
    image_path = os.path.abspath(image_path)
    mask_path = os.path.abspath(mask_path)

    # Iterating through each image in test data
    for n, i in tqdm(enumerate(test_ids), total=len(test_ids)):
        filename = i
        image_names.append(filename)
        i = os.path.join(image_path, i)

        # Creating an empty array of shape 1,512,512,3
        X = np.empty((1,512,512,3))
        # Read the image
        img = io.imread(i)
        # Resizing the image and converting them to array of type float64
        img = cv2.resize(img, (512,512))
        img = np.array(img, dtype=np.float64)
        
        # Standardising the image
        img /= 255.0
        # Converting the shape of image from 512,512,3 to 1,512,512,3
        X[0,] = img
        
        # Make prediction of mask
        for model_name, model in model_dict.items():
            start_time = time.time()
            prediction = model.predict(X, verbose=0)
            prediction = K.greater_equal(prediction, 0.5)
            prediction = np.round(prediction,0)
            end_time = time.time()
            eval_time = end_time - start_time
            #print(f'eval time for {model_name} is {eval_time}seconds')
            results_dict[model_name].append(prediction)
        
        image_id.append(os.path.join(image_path, filename))
        mask_id.append(os.path.join(mask_path, filename.split('.')[0] + '.png'))
        has_mask.append(1)
            
    return pd.DataFrame({'file_name': image_names, 'image_path': image_id, 
                         'mask_path': mask_id, 'has_mask': has_mask, **results_dict})

In [4]:
def iou_metric(y_true_in, y_pred_in):
    smooth = 1e-15
    intersection = y_true_in.ravel() * y_pred_in.ravel()
    union = y_true_in.ravel() + y_pred_in.ravel() - intersection

    iou = ((np.sum(intersection) + smooth)/(np.sum(union) + smooth))
    return iou

In [5]:
def show_visualization(df_pred, filename, model_dict, num_images):
    num_models = len(model_dict)
    fig, axs = plt.subplots(num_images, num_models + 1, figsize=(6 * (num_models + 1), 6 * num_images))
    W, H = 512, 512
    dim = (W, H)

    root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    filepath = os.path.join(root_dir, "results", filename)

    plt.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.9, wspace=0.1, hspace=0.3)

    for count, row in tqdm(enumerate(df_pred.itertuples())):
        if row.has_mask == 1 and count < num_images:
            img = cv2.imread(row.image_path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, dim)
            axs[count, 0].imshow(img)
            axs[count, 0].axis('off')
            axs[count, 0].set_title('Original Image ' + str(count + 1), fontsize=48, pad=10)

            """   mask = cv2.imread(row.mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, dim)
            img[mask == 255] = (200, 0, 0)
            axs[count, 1].imshow(img)
            axs[count, 1].axis('off')
            axs[count, 1].set_title('Ground Truth ' + str(count + 1), fontsize=48, pad=10)
            mask = np.array(mask, dtype=np.float64)
            mask = mask / 255 """

            for j, (model_name, model) in enumerate(model_dict.items()):
                pred = np.array(row._asdict()[model_name])
                #iou = iou_metric(mask, pred)
                pred = np.squeeze(pred)
                pred = np.uint8(pred > 0.6) *255
                img_model = cv2.imread(row.image_path, cv2.IMREAD_COLOR)
                img_model = cv2.resize(img_model, (512, 512))
                img_model = img_model.squeeze()
                img_model[pred == 255] = [(204, 204, 0), (102, 204, 0), (104, 0, 104), (0, 204, 204), (0, 153, 153)][j]
                axs[count, j + 1].imshow(img_model)
                axs[count, j + 1].axis('off')
                axs[count, j + 1].set_title(f'{model_name}', fontsize=42, pad=10)

    fig.tight_layout()
    plt.savefig(filepath, format='png', facecolor="w", transparent=False)
    plt.close()

## Load data for visualization

In [6]:
img_height = 512
img_width = 512
import random
root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
print(root_dir)
#path_test_levee = os.path.join(root_dir,  "datasets", 'original_dataset' ,"test")
path_test_levee = os.path.join(root_dir,  "datasets", "test_images", "levee_crack_test") #exp2_test, pothole_test, with_grassland, #no_grassland inside "test_images"

print(path_test_levee)

test_images_leveecrack = sorted(next(os.walk(path_test_levee + "/images"))[2])

random.Random(42).shuffle(test_images_leveecrack)



print(test_images_leveecrack)
image_leveecrack_path = path_test_levee + "/images"
mask_leveecrack_path = path_test_levee + "/masks"

/home/mpanta1/usace_workspace/sandboil_research
/home/mpanta1/usace_workspace/sandboil_research/datasets/test_images/levee_crack_test
['LCST16Jan18-006_20160124103700.jpg', 'pic_(16).jpg', 'pic_(4).jpg', 'LKBO15Jun26-001_20150803101000.jpg', 'pic_(8).jpg', '0171.png', 'LWMS11May11-006_20110522155200.jpg', '0021.png', '0102.png']


## Load Models

In [9]:
def load_model(model_name, custom_layer=False):
    result_folder_name = str(model_name)
    root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    model_path = os.path.join(root_dir, "models/state_of_the_art_comparison_bce_dice", str(model_name), "best_model.h5")
    tf.keras.backend.clear_session()
    
    if custom_layer:
        best_model = tf.keras.models.load_model(model_path, custom_objects={'PCALayer': PCALayer}, compile=False)
        
    else:
        print("here")
        best_model = tf.keras.models.load_model(model_path,compile=False)
    print(f'=========Loaded {model_name}===========')
    return best_model

In [13]:
M0 = load_model('unet_bce_dice_loss_new', False)
M1 = load_model('multiresunet_bce_dice_loss_new', False)
M2 = load_model('attentionunet_bce_dice_loss_new', False)
M3 = load_model('nestedunet_bce_dice_loss_new', False)
M4 = load_model('SandboilNet_negative_23_bce_dice_loss_new', True)

here
=========Loaded unet_bce_dice_loss_new===========
here
=========Loaded multiresunet_bce_dice_loss_new===========
here
=========Loaded attentionunet_bce_dice_loss_new===========
here
=========Loaded nestedunet_bce_dice_loss_new===========
=========Loaded SandboilNet_negative_23_bce_dice_loss_new===========


In [15]:
import time
import tensorflow.keras.backend as K
model_dict = {'M1':M0, 'M2':M1, 'M3':M2, 'M4':M3,'M5':M4 }
#model_dict = {'M1' : M4}

df_all_preds_all = []
df_all_preds_all = prediction(test_images_leveecrack, os.path.abspath(image_leveecrack_path), os.path.abspath(mask_leveecrack_path), model_dict)


  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:11<00:00,  1.32s/it]


In [18]:
num_images = 9
show_visualization(df_all_preds_all, "all_negative_images.png", model_dict, num_images)

9it [00:01,  5.06it/s]


: 

In [ ]:
df_for_visualization = df_all_preds_all.sample(frac = 0.98)
show_visualization(df_for_visualization, "four_models_inferences_98.png")